In [1]:
import ast
import os
import pandas as pd
import re

pd.set_option("display.max_columns", None)

os.chdir("../")

In [2]:
year = 2014
election_type = "general"

In [3]:
df = pd.read_csv("data/Belgium/2019_general.csv")\
    .rename(columns={"id": "polling_id"})
# df["electionLists"] = df["electionLists"].apply(lambda x: ast.literal_eval(x))
# df["labels"] = df["labels"].apply(lambda x: ast.literal_eval(x))
# df = df.explode("electionLists")
# df = df.explode("labels")

# A = pd.json_normalize(df["labels"])
# B = pd.json_normalize(df["electionLists"]).rename(columns={
#     "nrOfVotes": "value",
#     "partyLabel": "candidate"})

# df = df.join([A, B])

# print(list(df))

# df = df.drop(columns=["labels", "electionLists"])
df["year"] = df["electionDate"].str[0:4].astype(int)
df = df[df["year"] == year].copy()
# df = df.drop_duplicates()

df.head()
# df.shape

,polling_id,electionId,nrOfEligibleVoters,nrOfValidVotes,nrOfBlankVotes,nisCode,labels,description,electionType,electionDate,electionLists,year
0,228729,51199,0,13984,635,44001,"[{'language': 'French', 'label': 'Aalter'}, {'...",Municipality,Chamber,2014-05-25,"[{'id': 1309904, 'nrOfVotes': 1440, 'listNumbe...",2014
2,228582,51199,0,20532,1137,24001,"[{'language': 'French', 'label': 'Aarschot'}, ...",Municipality,Chamber,2014-05-25,"[{'id': 1308190, 'nrOfVotes': 1543, 'listNumbe...",2014
4,228458,51199,0,9934,279,11001,"[{'language': 'French', 'label': 'Aartselaar'}...",Municipality,Chamber,2014-05-25,"[{'id': 1306555, 'nrOfVotes': 738, 'listNumber...",2014
6,228581,51199,0,8968,311,23105,"[{'language': 'French', 'label': 'Affligem'}, ...",Municipality,Chamber,2014-05-25,"[{'id': 1308179, 'nrOfVotes': 437, 'listNumber...",2014
8,228788,51199,0,5958,623,52074,"[{'language': 'French', 'label': 'Aiseau-Presl...",Municipality,Chamber,2014-05-25,"[{'id': 1310812, 'nrOfVotes': 940, 'listNumber...",2014


In [4]:
len(df.polling_id.unique())

576

In [5]:
output = []
output_location = []
for polling_id, frag in df.groupby("polling_id"):
    # print(polling_id)

    parties = ast.literal_eval(frag["electionLists"].values[0])
    labels = ast.literal_eval(frag["labels"].values[0])

    commune = labels[1]["label"]

    output_location.append({
        "commune": commune,
        "polling_id": polling_id
    })

    for party in parties:
        # print(party)
        output.append({
            "polling_id": polling_id,
            "value": party["nrOfVotes"],
            "candidate": party["partyLabel"],
            "commune": commune
        })

df_location = pd.DataFrame(output_location)
df_filtered = pd.DataFrame(output)
df_filtered.head()

,polling_id,value,candidate,commune
0,228458,738,GROEN,Aartselaar
1,228458,546,VLAAMS BELANG,Aartselaar
2,228458,1350,OpenVld,Aartselaar
3,228458,773,SP.A,Aartselaar
4,228458,4724,N-VA,Aartselaar


In [6]:
tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt.head()

,polling_id,candidate,value,rate
0,228458,B.U.B.-Belg.Unie,10,0.001007
1,228458,CD&V,1488,0.149789
2,228458,GROEN,738,0.074290
3,228458,N-VA,4724,0.475539
4,228458,OpenVld,1350,0.135897


In [7]:
tt.to_csv(f"data_output/Belgium/{year}_{election_type}.csv.gz", compression="gzip", index=False)

In [15]:
df_lau = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/501971/EU-27-LAU-2022-NUTS-2021.xlsx",
    sheet_name="BE"
)
df_lau = df_lau[["LAU NAME NATIONAL", "NUTS 3 CODE", "LAU CODE"]].rename(columns={"LAU NAME NATIONAL": "commune", "NUTS 3 CODE": "nuts_3", "LAU CODE": "lau_id"})
df_lau["nuts_2"] = df_lau["nuts_3"].str[0:4]
df_lau["nuts_1"] = df_lau["nuts_3"].str[0:3]
df_lau["lau_id"] = df_lau["lau_id"].astype(str).str.zfill(8)
df_lau["commune"] = df_lau["commune"].str.replace("’", "'")
df_lau["commune"] = df_lau["commune"].apply(lambda x: re.sub(r"\([^)]*\)", "", x)).str.strip()

if year == 2019:
    df_lau["commune"] = df_lau["commune"].replace({
        "'s Gravenbrakel": "s Gravenbrakel",
        "Celles": "Celles (lez-Tournai)",
        "Eigenbrakel": "Braine-l'Alleud",
        "Meix-devant-Virton": "Meix-Devant-Virton",
        "Opzullik": "Silly",
        "Saint-Nicolas": "Saint-Nicolas (Liège)",
        "Sint-Joost-ten-Node": "Sint-Joost-Ten-Node",
        "Villers-Le-Bouillet": "Villers-le-Bouillet"
    })
elif year == 2014:
    df_lau["commune"] = df_lau["commune"].replace({
        "'s Gravenbrakel": "s Gravenbrakel",
        "Celles": "Celles (lez-Tournai)",
        "Eigenbrakel": "Braine-l'Alleud",
        "Meix-devant-Virton": "Meix-Devant-Virton",
        "Opzullik": "Silly",
        "Sint-Joost-ten-Node": "Sint-Joost-Ten-Node",
        "Villers-Le-Bouillet": "Villers-le-Bouillet",
        'Aiseau-Presles': "Aiseau-Presles Nl",
        "De Haan": "de Haan",
        "Glabbeek": "Glabeek",
        "Kruisem": "Kruisem",
        "La Bruyère": "la Bruyère",
        "Lievegem": "Lievegem",
        "Montigny-le-Tilleul": "Montigne-le-Tilleul",
        "Oudsbergen": "Oudsbergen",
        "Pelt": "Pelt",
        "Puurs-Sint-Amands": "Puurs-Sint-Amands",
        "Saint-Nicolas": "Saint-Nicolas (liège)"
    })

df_lau.head()

,commune,nuts_3,lau_id,nuts_2,nuts_1
0,Anderlecht,BE100,00021001,BE10,BE1
1,Oudergem,BE100,00021002,BE10,BE1
2,Sint-Agatha-Berchem,BE100,00021003,BE10,BE1
3,Brussel,BE100,00021004,BE10,BE1
4,Etterbeek,BE100,00021005,BE10,BE1


In [16]:
set(df_lau.commune) - set(df_location.commune)

{'Kruisem', 'Lievegem', 'Oudsbergen', 'Pelt', 'Puurs-Sint-Amands'}

In [18]:
set(df_location.commune) - set(df_lau.commune)

set()

In [10]:
df_location = pd.merge(df_location, df_lau, on="commune", how="left")

df_location.to_csv(f"data_output/Belgium/{year}_{election_type}_location.csv.gz", compression="gzip", index=False)